In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
    
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
!find . -name '._DS_Store' -type f -delete

In [2]:
batch_size = 32
img_height = 180
img_width = 180

In [26]:
import os

# Specify the directory path
dir_path = "/kaggle/input/cat-and-dog/training_set/training_set"

# Specify the file name
file_name = "_DS_Store"

# Walk through the directory
for dirname, _, _ in os.walk(dir_path):
    # Construct the file path
    file_path = os.path.join(dirname, file_name)
    
    # Check if the file exists
    if os.path.isfile(file_path):
        # Remove the file
        os.remove(file_path)
        print(f"Removed file: {file_path}")
    else:
        print(f"No such file: {file_path}")

No such file: /kaggle/input/cat-and-dog/training_set/training_set/_DS_Store


OSError: [Errno 30] Read-only file system: '/kaggle/input/cat-and-dog/training_set/training_set/dogs/_DS_Store'

In [19]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  "/kaggle/input/cat-and-dog/training_set/training_set/",
  validation_split=0.2,
  labels='inferred',
  label_mode = None,
  subset="training",
  seed=123,
  class_names = ['cats', 'dogs'],
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 8005 files belonging to 1 classes.
Using 6404 files for training.


In [13]:
print(train_ds.class_names)

['']


In [10]:
train_ds.take(1)

<_TakeDataset element_spec=TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None)>

In [7]:
import matplotlib.pyplot as plt
fig = plt.figure()

for image, label in train_ds.take(1):
  plt.subplot(4,4, 1)
  plt.imshow(image[0])

ValueError: too many values to unpack (expected 2)

<Figure size 640x480 with 0 Axes>